In [1]:
!pip install transformers datasets evaluate

In [2]:
import csv
import evaluate
import pandas as pd

2025-11-17 05:42:04.321274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763358124.344597     376 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763358124.351562     376 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# from huggingface_hub import login
# login()

In [4]:
# Verifica GPU disponible en tu notebook
!nvidia-smi

# En código
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memoria total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Memoria disponible: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB usados")


Mon Nov 17 05:42:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             25W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
HF_MODEL = 'HuggingFaceTB/SmolLM2-1.7B'
QWEN_MODEL = 'Qwen/Qwen2.5-1.5B'

In [6]:
perplexity = evaluate.load("perplexity", module_type="metric")

def calculate_reprompt_perplexity(texts: list[str], model: str) -> list[float]:    
    result = perplexity.compute(
        model_id=model,
        predictions=texts,
        add_start_token=False
    )
    return result['perplexities']


def read_reprompts_file(filepath: str):
    with open(filepath, 'r') as file_:
        reader = csv.DictReader(file_)
        items = [item for item in reader]
    return items

## GPT5-Nano Perplexity

In [7]:
reprompt_items = read_reprompts_file(
    '/kaggle/input/gpt5-nano-reprompts-v3/pipeline_results_gpt5_nano_50_prompt_v3.csv'
)

pp_items = []
for model in (HF_MODEL, QWEN_MODEL, ):
    for item, res in zip(
        reprompt_items, 
        calculate_reprompt_perplexity(
            [item['reprompt'] for item in reprompt_items],
            model
        )):
        pp_items.append(item | {'model': model, 'perplexity': res})


df_nano = pd.DataFrame(pp_items)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
df_nano['id_prompt'] = df_nano['id_prompt'].astype(int)
df_nano['rank_pp'] = df_nano.groupby('id_prompt')['perplexity'].transform(lambda x: x.rank(ascending=True))

df_rank_nano = df_nano[df_nano['rank_pp'] == 1].copy()
df_rank_nano['prompt_source'] = 'gpt5-nano'
df_rank_nano

,id_prompt,prompt,reprompt,model,perplexity,rank_pp,prompt_source
0,1,I'm at the park and I'm gonna eat a turkey san...,Lead with a harmonium melody that embodies joy...,HuggingFaceTB/SmolLM2-1.7B,35.991936,1.0,gpt5-nano
1,2,I'm in the cafe and I'm gonna drink a cappucci...,Lead instrument: a bright synth lead to align ...,HuggingFaceTB/SmolLM2-1.7B,45.524662,1.0,gpt5-nano
2,3,We're at the beach and we're gonna eat two fis...,Lead with a bright acoustic guitar as the melo...,HuggingFaceTB/SmolLM2-1.7B,55.541813,1.0,gpt5-nano
3,4,I'm in the library and I'm gonna drink a cup o...,"Lead with a bright flute to embody a sweet, ha...",HuggingFaceTB/SmolLM2-1.7B,44.198875,1.0,gpt5-nano
4,5,We're at the stadium and we're gonna eat a chi...,Lead with a bright warm synth lead to carry a ...,HuggingFaceTB/SmolLM2-1.7B,59.211075,1.0,gpt5-nano
5,6,I'm in the kitchen and I'm gonna drink a glass...,"Lead the melody on a piano with a bright, live...",HuggingFaceTB/SmolLM2-1.7B,36.174324,1.0,gpt5-nano
6,7,I'm at the airport gate and I'm gonna eat a ch...,Lead with harmonium melody that channels joyfu...,HuggingFaceTB/SmolLM2-1.7B,61.695560,1.0,gpt5-nano
7,8,We're on the train and we're gonna drink two b...,Lead with a flute melody that channels sweetne...,HuggingFaceTB/SmolLM2-1.7B,46.513947,1.0,gpt5-nano
8,9,I'm in the gym and I'm gonna drink a vanilla p...,"Lead with a harmonium melody, aligning with ch...",HuggingFaceTB/SmolLM2-1.7B,38.228275,1.0,gpt5-nano
9,10,I'm at the bakery and I'm gonna eat a chocolat...,Lead with a mellow marimba melody that embodie...,HuggingFaceTB/SmolLM2-1.7B,31.683151,1.0,gpt5-nano


## Kimi-K2 Perplexity

In [9]:
reprompt_items = read_reprompts_file(
    '/kaggle/input/kimi-k2-reprompts-v3/pipeline_results_kimi_k2_thinking_50_prompt_v3.csv'
)

pp_items = []
for model in (HF_MODEL, QWEN_MODEL, ):
    for item, res in zip(
        reprompt_items, 
        calculate_reprompt_perplexity(
            [item['reprompt'] for item in reprompt_items],
            model
        )):
        pp_items.append(item | {'model': model, 'perplexity': res})


df_kimi = pd.DataFrame(pp_items)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
df_kimi['id_prompt'] = df_kimi['id_prompt'].astype(int)
df_kimi['rank_pp'] = df_kimi.groupby('id_prompt')['perplexity'].transform(lambda x: x.rank(ascending=True))

df_rank_kimi = df_kimi[df_kimi['rank_pp'] == 1].copy()
df_rank_kimi['prompt_source'] = 'kimi_k2'
df_rank_kimi

,id_prompt,prompt,reprompt,model,perplexity,rank_pp,prompt_source
0,1,I'm at the park and I'm gonna eat a turkey san...,"A warm, reedy harmonium leads with a savory, s...",HuggingFaceTB/SmolLM2-1.7B,73.755463,1.0,kimi_k2
1,2,I'm in the cafe and I'm gonna drink a cappucci...,"A bright, moderately-pitched electric guitar d...",HuggingFaceTB/SmolLM2-1.7B,49.680096,1.0,kimi_k2
2,3,We're at the beach and we're gonna eat two fis...,"Mandolin's bright, sweet timbre articulates a ...",HuggingFaceTB/SmolLM2-1.7B,56.160625,1.0,kimi_k2
3,4,I'm in the library and I'm gonna drink a cup o...,"A classical guitar introduces a serene, sweet-...",HuggingFaceTB/SmolLM2-1.7B,55.345802,1.0,kimi_k2
4,5,We're at the stadium and we're gonna eat a chi...,Bright warm synths lead an upbeat melody throu...,HuggingFaceTB/SmolLM2-1.7B,85.010170,1.0,kimi_k2
5,6,I'm in the kitchen and I'm gonna drink a glass...,A sweet-sounding harmonium melody projects war...,HuggingFaceTB/SmolLM2-1.7B,67.825134,1.0,kimi_k2
6,7,I'm at the airport gate and I'm gonna eat a ch...,"A glockenspiel's bright, metallic timbre dance...",HuggingFaceTB/SmolLM2-1.7B,30.258955,1.0,kimi_k2
8,9,I'm in the gym and I'm gonna drink a vanilla p...,"A sweet-sounding harmonium leads with warm, br...",HuggingFaceTB/SmolLM2-1.7B,71.533539,1.0,kimi_k2
9,10,I'm at the bakery and I'm gonna eat a chocolat...,A mellow marimba leads with a sweet-sounding m...,HuggingFaceTB/SmolLM2-1.7B,35.487682,1.0,kimi_k2
10,11,We're on the rooftop terrace and we're gonna d...,"A bright glockenspiel leads with a sweet, joyf...",HuggingFaceTB/SmolLM2-1.7B,31.616755,1.0,kimi_k2


## Merge Results to Compare GPT5-Nano vs Kimi-K2

In [11]:
concat_df = pd.concat([df_rank_nano, df_rank_kimi])
concat_df['rank'] = concat_df.groupby('id_prompt')['perplexity'].transform(lambda x: x.rank(ascending=True))
concat_df

,id_prompt,prompt,reprompt,model,perplexity,rank_pp,prompt_source,rank
0,1,I'm at the park and I'm gonna eat a turkey san...,Lead with a harmonium melody that embodies joy...,HuggingFaceTB/SmolLM2-1.7B,35.991936,1.0,gpt5-nano,1.0
1,2,I'm in the cafe and I'm gonna drink a cappucci...,Lead instrument: a bright synth lead to align ...,HuggingFaceTB/SmolLM2-1.7B,45.524662,1.0,gpt5-nano,1.0
2,3,We're at the beach and we're gonna eat two fis...,Lead with a bright acoustic guitar as the melo...,HuggingFaceTB/SmolLM2-1.7B,55.541813,1.0,gpt5-nano,1.0
3,4,I'm in the library and I'm gonna drink a cup o...,"Lead with a bright flute to embody a sweet, ha...",HuggingFaceTB/SmolLM2-1.7B,44.198875,1.0,gpt5-nano,1.0
4,5,We're at the stadium and we're gonna eat a chi...,Lead with a bright warm synth lead to carry a ...,HuggingFaceTB/SmolLM2-1.7B,59.211075,1.0,gpt5-nano,1.0
...,...,...,...,...,...,...,...,...
49,50,I feel energetic and I'm gonna drink an iced A...,"A bright ukulele leads with a crisp, mid-to-hi...",HuggingFaceTB/SmolLM2-1.7B,42.658844,1.0,kimi_k2,2.0
57,8,We're on the train and we're gonna drink two b...,"A marimba's sweet, resonant timbre should arti...",Qwen/Qwen2.5-1.5B,62.259369,1.0,kimi_k2,2.0
64,15,I'm in the hotel lobby and I'm gonna drink a l...,"A cello leads with a velvety, amber-hued melod...",Qwen/Qwen2.5-1.5B,48.667721,1.0,kimi_k2,2.0
90,41,I feel bored and I'm gonna eat a chocolate chi...,A warm-toned acoustic guitar leads with a swee...,Qwen/Qwen2.5-1.5B,50.411373,1.0,kimi_k2,1.0


In [12]:
concat_df[concat_df['rank'] == 1].groupby('prompt_source').mean('perplexity')['perplexity']

prompt_source
gpt5-nano    41.671094
kimi_k2      40.209906
Name: perplexity, dtype: float64

## Conclusiones

- Se calculó la métrica perplexity para ambos reprompts originados de los modelos GPT5-Nano y Kimi-K2 usando los modelos `HuggingFaceTB/SmolLM2-1.7B` y `Qwen/Qwen2.5-1.5B`. Los modelos fueron elegidos debido a la factibilidad de poder ser usados en los ambientes de GPU disponibles (Kaggle).
- Despues de calcular la métrica usando los anteriores modelos de forma separada para los dos origenes de reprompt se identiica que el modelo `HuggingFaceTB/SmolLM2-1.7B` obtiene un valor menor para la métrica. Se mantiene en cada uno de las colecciones de reprompts los items con menor valor de perplexity.
- Se unen los resultados con menor perplexity en un solo conjunto de referencia y se filtra para mantener de nuevo la menor perplejidad para identificar el mejor prompt_source. Se calcula la perplejidad promedio de los items de cada prompt source. La menor perplejidad para reprompt es usando el modelo `kimi-k2`. **Nota**: Es importante resaltar que los prompts originales fueron los mismos en las colecciones.
- A través de inspeccion visual, se habia identificado que ambas fuentes de reprompt usan los descriptores cross-modales y musicales recuperados a través del RAG, sin embargo los reprompt del modelo `kimi-k2` son diversos y coherentes en la intención de generación.